In [1]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 59.4 MB/s eta 0:00:00


In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig, TrainingArguments, Trainer

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base-v2022")

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


- 기쁨 : 0 
- 슬픔 : 1 
- 분노 : 2
- 불안 : 3
- 당황 : 4
- 상처 : 5


In [4]:
train_df = pd.read_csv('/content/drive/MyDrive/sentiment_kcelectra/감성대화말뭉치_Training.csv')
train_df

,id,text,labels
0,1,일은 왜 해도 해도 끝이 없을까? 화가 난다.,2
1,2,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,2
2,3,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,2
3,4,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,2
4,5,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,2
...,...,...,...
51625,51626,나이가 먹고 이제 돈도 못 벌어 오니까 어떻게 살아가야 할지 막막해. 능력도 없고.,2
51626,51627,몸이 많이 약해졌나 봐. 이제 전과 같이 일하지 못할 것 같아 너무 짜증 나.,3
51627,51628,이제 어떻게 해야 할지 모르겠어. 남편도 그렇고 노후 준비도 안 되어서 미래가 걱정돼.,5
51628,51629,몇십 년을 함께 살았던 남편과 이혼했어. 그동안의 세월에 배신감을 느끼고 너무 화가 나.,3


In [5]:
test_df = pd.read_csv('/content/drive/MyDrive/sentiment_kcelectra/감성대화말뭉치_Test.csv')
test_df

,id,text,labels
0,1,이번 프로젝트에서 발표를 하는데 내가 실수하는 바람에 우리 팀이 감점을 받았어. 너...,3
1,2,회사에서 중요한 프로젝트를 혼자 하게 됐는데 솔직히 두렵고 무서워.,3
2,3,상사가 너무 무섭게 생겨서 친해지는 게 너무 두려워.,3
3,4,이번에 힘들게 들어간 첫 직장이거든. 첫 직장이라서 그런지 너무 긴장된다.,3
4,5,직장에서 동료들이랑 관계가 안 좋아질까 봐 걱정돼.,3
...,...,...,...
6636,6637,나랑 비슷한 시기에 결혼하는 친구는 시댁에서 집을 해줘서 너무 부러워. 우리는 대출...,4
6637,6638,친구 한 명이 결혼해서 아이를 가졌는데 너무 행복해 보이더라. 기분이 좋지만은 않아.,4
6638,6639,남들은 결혼 전에 일억을 모았다는데 난 뭐를 한 것인지 모르겠어. 자괴감만 드네.,4
6639,6640,나보다 결혼을 먼저 한 친구가 부러워. 그 친구 남편은 직장도 내 남편보다 좋고 키...,4


In [6]:
tokenized_train_sentences = tokenizer(
    list(train_df["text"]),
    return_tensors= "pt",        # pytorch의 tensor 형태로 return
    max_length = 128,            # 최대 토큰 길이
    padding = True,              # 제로패팅
    truncation = True,           # 최대 토큰 길이 초과하면 자름
    add_special_tokens = True,   # special token 추가
)

In [7]:
print(tokenized_train_sentences[0])
print(tokenized_train_sentences[0].tokens)
print(tokenized_train_sentences[0].ids)
print(tokenized_train_sentences[0].attention_mask)

Encoding(num_tokens=64, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
['[CLS]', '일은', '왜', '해도', '해도', '끝이', '없을까', '?', '화가', '난다', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
[2, 10393, 2607, 8487, 8487, 12021, 23573, 33, 10540, 10764, 18, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [8]:
tokenized_test_sentences = tokenizer(
    list(test_df["text"]),
    return_tensors= "pt",        # pytorch의 tensor 형태로 return
    max_length = 128,            # 최대 토큰 길이
    padding = True,              # 제로패팅
    truncation = True,           # 최대 토큰 길이 초과하면 자름
    add_special_tokens = True,   # special token 추가가
)

In [9]:
print(tokenized_test_sentences[0])
print(tokenized_test_sentences[0].tokens)
print(tokenized_test_sentences[0].ids)
print(tokenized_test_sentences[0].attention_mask)

Encoding(num_tokens=42, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
['[CLS]', '이번', '프로젝트', '##에서', '발표를', '하는데', '내가', '실수', '##하는', '바람에', '우리', '팀', '##이', '감', '##점을', '받았', '##어', '.', '너무', '미안해', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
[2, 8156, 36780, 7938, 32670, 8473, 8201, 11029, 7934, 20830, 7955, 3351, 4034, 868, 12030, 9491, 4092, 18, 8024, 21941, 18, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [10]:
class SentimentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key : torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [11]:
train_label = train_df["labels"].values
test_label = test_df["labels"].values

train_dataset = SentimentDataset(tokenized_train_sentences, train_label)
test_dataset = SentimentDataset(tokenized_test_sentences, test_label)

In [28]:
config = AutoConfig.from_pretrained('beomi/KcELECTRA-base-v2022')
config.hidden_dropout_prob = 0.2
config.num_labels = 6

model = AutoModelForSequenceClassification.from_pretrained("beomi/KcELECTRA-base-v2022", config=config)
model.to(device)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--beomi--KcELECTRA-base-v2022/snapshots/4431b6c7ad00f82fd50880864574cef97e0a368b/config.json
Model config ElectraConfig {
  "_name_or_path": "beomi/KcELECTRA-base-v2022",
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "tokenizer_class": "BertTokenizer",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(54343, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.2, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm

In [32]:
args = TrainingArguments(
    output_dir='/content/drive/MyDrive/sentiment_kcelectra/',           # 학습결과 저장경로
    num_train_epochs=5,                                                 # 학습 에포크 설정
    per_device_train_batch_size=32,                                     # 학습 배치 사이즈 설정
    per_device_eval_batch_size=64,                                      # 테스트 배치 사이즈 설정  
    logging_dir='/content/drive/MyDrive/sentiment_kcelectra/logs',      # 학습 log 저장 경로
    logging_steps=500,                                                  # 학습 log 기록 단위
    save_total_limit=7,                                                 # 학습 결과 저장 최대 갯수
    resume_from_checkpoint=True
)   

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [24]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

In [25]:
# 평가지표를 위한 함수
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy' : acc,
        'f1' : f1,
        'precision' : precision,
        'recall' : recall
    }

In [33]:
trainer = Trainer(
    model = model,
    args = args,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    compute_metrics=compute_metrics,
)

In [34]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 51630
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 8070
  Number of trainable parameters = 127781382
<ipython-input-10-8f22ac0f3218>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key : torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss
500,1.149600
1000,1.091000
1500,1.110100
2000,1.039700
2500,1.003500
3000,1.009100
3500,0.939300
4000,0.890800
4500,0.882500
5000,0.833200


Saving model checkpoint to /content/drive/MyDrive/sentiment_kcelectra/checkpoint-500
Configuration saved in /content/drive/MyDrive/sentiment_kcelectra/checkpoint-500/config.json
Model weights saved in /content/drive/MyDrive/sentiment_kcelectra/checkpoint-500/pytorch_model.bin
<ipython-input-10-8f22ac0f3218>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key : torch.tensor(val[idx]) for key, val in self.encodings.items()}
Saving model checkpoint to /content/drive/MyDrive/sentiment_kcelectra/checkpoint-1000
Configuration saved in /content/drive/MyDrive/sentiment_kcelectra/checkpoint-1000/config.json
Model weights saved in /content/drive/MyDrive/sentiment_kcelectra/checkpoint-1000/pytorch_model.bin
<ipython-input-10-8f22ac0f3218>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach

TrainOutput(global_step=8070, training_loss=0.8842216609874475, metrics={'train_runtime': 3053.8598, 'train_samples_per_second': 84.532, 'train_steps_per_second': 2.643, 'total_flos': 8490569790297600.0, 'train_loss': 0.8842216609874475, 'epoch': 5.0})

In [35]:
trainer.evaluate(eval_dataset=test_dataset)

***** Running Evaluation *****
  Num examples = 6641
  Batch size = 64
<ipython-input-10-8f22ac0f3218>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key : torch.tensor(val[idx]) for key, val in self.encodings.items()}


Trainer is attempting to log a value of "[0.93860387 0.66921972 0.7295898  0.69859919 0.67835149 0.63737374]" of type <class 'numpy.ndarray'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.95793991 0.64364641 0.73046252 0.70272727 0.67356538 0.64850976]" of type <class 'numpy.ndarray'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.92003298 0.69690927 0.72871917 0.69451932 0.68320611 0.6266137 ]" of type <class 'numpy.ndarray'> for key "eval/recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.8448866009712219,
 'eval_accuracy': 0.730462279777142,
 'eval_f1': array([0.93860387, 0.66921972, 0.7295898 , 0.69859919, 0.67835149,
        0.63737374]),
 'eval_precision': array([0.95793991, 0.64364641, 0.73046252, 0.70272727, 0.67356538,
        0.64850976]),
 'eval_recall': array([0.92003298, 0.69690927, 0.72871917, 0.69451932, 0.68320611,
        0.6266137 ]),
 'eval_runtime': 16.8898,
 'eval_samples_per_second': 393.195,
 'eval_steps_per_second': 6.158,
 'epoch': 5.0}

In [21]:
def sentence_predict(sent):
    # 평가모드로 변경
    model.eval()

    # 입력된 문장 토크나이징
    tokenized_sent = tokenizer(
        sent,
        return_tensors="pt",
        truncation=True,
        add_special_tokens=True,
        max_length=128
    )

    # 모델이 위치한 GPU로 이동
    tokenized_sent.to(device)

    # 예측
    with torch.no_grad():
        outputs = model(
            input_ids = tokenized_sent["input_ids"],
            attention_mask=tokenized_sent["attention_mask"],
            token_type_ids=tokenized_sent["token_type_ids"]
        )

    # 결과 return
    logits = outputs[0]
    logits = logits.detach().cpu()
    prob = logits.softmax(dim=1)
    print(prob)
    result = logits.argmax(-1)
    if result == 0:
        result = "기쁨 ㅎㅎ"
    elif result == 1:
        result = "슬픔 ㅠㅠ"
    elif result == 2:
        result = "분노 --"
    elif result == 3: 
        result = "불안... ㄷㄷㄷ"
    elif result == 4:
        result = "당황?!"
    elif result == 5:
        result = "상처...ㅠㅠㅠ"
    return result

#0 입력시 종료
while True:
     sentence = input("문장을 입력해주세요: ")
     if sentence == "0": 
         break
     print(sentence_predict(sentence))
     print("\n")
    

문장을 입력해주세요: 제발 집에 보내줘
tensor([[0.0398, 0.0333, 0.8742, 0.0340, 0.0068, 0.0119]])
분노 --


문장을 입력해주세요: 안보내줘?
tensor([[1.6231e-03, 1.8477e-03, 9.8900e-01, 4.7020e-03, 9.5466e-04, 1.8687e-03]])
분노 --


문장을 입력해주세요: 시발 장난해
tensor([[0.0649, 0.0463, 0.7761, 0.0168, 0.0039, 0.0921]])
분노 --


문장을 입력해주세요: 어떻게 그런말을 할수가 있어
tensor([[0.0529, 0.1073, 0.4714, 0.0128, 0.0148, 0.3410]])
분노 --


문장을 입력해주세요: 실망이야
tensor([[7.6892e-04, 9.7818e-01, 1.1300e-02, 2.5673e-03, 3.3673e-03, 3.8163e-03]])
슬픔 ㅠㅠ


문장을 입력해주세요: 집에 언제가?
tensor([[0.1157, 0.1666, 0.2102, 0.4165, 0.0187, 0.0723]])
불안... ㄷㄷㄷ


문장을 입력해주세요: 이건 좀 아닌듯..
tensor([[0.0128, 0.3400, 0.1821, 0.4024, 0.0183, 0.0445]])
불안... ㄷㄷㄷ




KeyboardInterrupt: ignored